In [1]:
# Run this at the start
import xml.etree.ElementTree as ET
from ultralytics import YOLO
from pathlib import Path
import random
import shutil
import cv2
import os
import shutil
from ultralytics import YOLO

In [ ]:
yolo_folder = '/home/student/Desktop/current_model_ai/obj_train_data'               # Yolo1.1 export from Cvat
images_folder = '/home/student/Desktop/current_model_ai/ImagesTotal'                # Image folder
output_folder = '/home/student/Desktop/current_model_ai'                            # Output folder

# Output directories and clear existing labels
for subset in ['train', 'val', 'test']:
    images_path = os.path.join(output_folder, subset, 'images')
    labels_path = os.path.join(output_folder, subset, 'labels')

    os.makedirs(images_path, exist_ok=True)
    os.makedirs(labels_path, exist_ok=True)

    # Clear 
    for file in os.listdir(labels_path):
        if file.endswith(".timport os
import shutil
from ultralytics import YOLOxt"):
            os.remove(os.path.join(labels_path, file))

# Collect annotation and their corresponding image
data_pairs = []

print("Searching for matching images and labels...")
for label_file in os.listdir(yolo_folder):
    if label_file.endswith(".txt"):
        label_path = os.path.join(yolo_folder, label_file)
        
        # Check for corresponding image 
        image_found = False
        for ext in ['.png', '.jpg', '.jpeg']:
            img_name = label_file.replace('.txt', ext)
            src_img = os.path.join(images_folder, img_name)
            if os.path.exists(src_img):
                data_pairs.append((src_img, label_path))
                print(f"Found: {img_name} with {label_file}")
                image_found = True
                break
        if not image_found:
            print(f"No matching image for label: {label_file}")

print(f"Total pairs found: {len(data_pairs)}")

# Shuffle and split data
random.shuffle(data_pairs)
num_images = len(data_pairs)
train_split = int(0.7 * num_images)
val_split = int(0.9 * num_images)

train_data = data_pairs[:train_split]
val_data = data_pairs[train_split:val_split]
test_data = data_pairs[val_split:]

# Save images and annotations in split
def save_data(data, subset):
    print(f"Saving {subset} data...")
    saved_count = 0
    for img_path, label_path in data:
        # Get filenames
        img_name = os.path.basename(img_path)
        label_name = os.path.basename(label_path)

        # Set destination paths
        dst_img = os.path.join(output_folder, subset, 'images', img_name)
        dst_label = os.path.join(output_folder, subset, 'labels', label_name)

        # Copy image and label
        shutil.copy(img_path, dst_img)
        shutil.copy(label_path, dst_label)
        saved_count += 1

    print(f"{saved_count} images and labels saved in {subset}.")

# Save to each subset
save_data(train_data, 'train')
save_data(val_data, 'val')
save_data(test_data, 'test')

print(f"Data split and saved successfully.")



Searching for matching images and labels...
Found: rotated_180_387.png with rotated_180_387.txt
Found: rotated_270_85.png with rotated_270_85.txt
Found: rotated_90_343.png with rotated_90_343.txt
Found: rotated_180_51.png with rotated_180_51.txt
Found: rotated_90_166.png with rotated_90_166.txt
Found: rotated_180_350.png with rotated_180_350.txt
Found: rotated_180_232.png with rotated_180_232.txt
Found: rotated_90_323.png with rotated_90_323.txt
Found: rotated_90_245.png with rotated_90_245.txt
Found: rotated_90_203.png with rotated_90_203.txt
Found: Flipped_horizontal_64.png with Flipped_horizontal_64.txt
Found: rotated_270_99.png with rotated_270_99.txt
Found: rotated_270_409.png with rotated_270_409.txt
Found: rotated_270_167.png with rotated_270_167.txt
Found: rotated_90_336.png with rotated_90_336.txt
Found: rotated_270_387.png with rotated_270_387.txt
Found: rotated_270_162.png with rotated_270_162.txt
Found: rotated_180_355.png with rotated_180_355.txt
Found: rotated_270_415.png

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.ndimage import gaussian_filter1d

# Path to results.csv (auto-saved by YOLOv8 during training)
results_path = '/home/student/Desktop/current_model_ai/folder_train/folds_retrain/results.csv'

# Output folder for plots
plot_dir = os.path.join(os.path.dirname(results_path), 'combined_plots')
os.makedirs(plot_dir, exist_ok=True)

# Load the results
df = pd.read_csv(results_path)

# Apply Gaussian smoothing for clean curves
def smooth(series, sigma=1.5):
    return gaussian_filter1d(series, sigma=sigma)

# Plot pairs
plot_pairs = {
    'Box Loss': ('train/box_loss', 'val/box_loss'),
    'Cls Loss': ('train/cls_loss', 'val/cls_loss'),
    'DFL Loss': ('train/dfl_loss', 'val/dfl_loss'),
    'mAP50': ('metrics/mAP50(B)', None),
    'mAP50-95': ('metrics/mAP50-95(B)', None)
}

# Create plots
for title, (train_col, val_col) in plot_pairs.items():
    plt.figure()
    x = df.index

    if train_col in df.columns:
        plt.plot(x, smooth(df[train_col]), label='Train', linestyle='--')

    if val_col and val_col in df.columns:
        plt.plot(x, smooth(df[val_col]), label='Val', linestyle='-')

    plt.title(f'{title} (Smoothed)')
    plt.xlabel('Epoch')
    plt.ylabel(title)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, f'{title.replace(" ", "_").lower()}.png'))
    plt.close()

print(f"✅ Combined plots saved in: {plot_dir}")


✅ Combined plots saved in: /home/student/Desktop/current_model_ai/folder_train/folds_retrain/combined_plots


In [6]:
import os
import shutil
from ultralytics import YOLO
import numpy as np
import cv2

# === Setup ===
train_folder = "/home/student/Desktop/current_model_ai/folder_train"
image_dir = '/home/student/Desktop/current_model_ai/val/images'  # Folder with .jpg images
output_fp_dir = 'false_positive_visuals'
os.makedirs(output_fp_dir, exist_ok=True)
# Remove the folder if it exists
if os.path.exists(train_folder):
    shutil.rmtree(train_folder)

# Load the model
model = YOLO('yolo11n.pt')

# === Training ===
model.train(
    data='/home/student/Desktop/current_model_ai/data.yaml',
    epochs=30,
    imgsz=640,
    batch=16,
    project='/home/student/Desktop/current_model_ai/folder_train',
    name='folds_retrain',
    shear=5.0,
    scale=0.1,
    translate=0.0,
    degrees=15.0,
    augment=True
)

# === Validation ===
val_results = model.val(data='/home/student/Desktop/current_model_ai/data.yaml', save_txt=True)

# Automatically detect where YOLO saved predictions
pred_dir = os.path.join(val_results.save_dir, 'labels')
gt_dir = '/home/student/Desktop/current_model_ai/val/labels'

# === Helper Functions ===
def yolo_to_xyxy(line, img_w=640, img_h=640):
    cls, x, y, w, h = map(float, line)
    x1 = (x - w / 2) * img_w
    y1 = (y - h / 2) * img_h
    x2 = (x + w / 2) * img_w
    y2 = (y + h / 2) * img_h
    return int(cls), [x1, y1, x2, y2]

def center_distance(box1, box2):
    cx1, cy1 = (box1[0] + box1[2]) / 2, (box1[1] + box1[3]) / 2
    cx2, cy2 = (box2[0] + box2[2]) / 2, (box2[1] + box2[3]) / 2
    return np.linalg.norm([cx1 - cx2, cy1 - cy2])

def box_iou(box1, box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH
    box1Area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2Area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    unionArea = box1Area + box2Area - interArea
    return interArea / unionArea if unionArea > 0 else 0.0

# === Evaluation ===
total_distance = 0
count = 0
fold_detections = 0
total_fold_ground_truth = 0
false_positive_images = []
background_images = []

for label_file in os.listdir(gt_dir):
    gt_path = os.path.join(gt_dir, label_file)
    pred_path = os.path.join(pred_dir, label_file)
    if not os.path.exists(pred_path):
        continue

    with open(gt_path, 'r') as f:
        gt_lines = [line.strip().split() for line in f.readlines()]
    with open(pred_path, 'r') as f:
        pred_lines = [line.strip().split() for line in f.readlines()]

    gt_boxes = [yolo_to_xyxy(line) for line in gt_lines if int(line[0]) == 0]
    pred_boxes = [yolo_to_xyxy(line) for line in pred_lines if int(line[0]) == 0]

    matched_preds = set()
    matched_gts = set()

    for i, (gt_cls, gt_box) in enumerate(gt_boxes):
        best_iou = 0
        best_j = -1
        for j, (pred_cls, pred_box) in enumerate(pred_boxes):
            if j in matched_preds:
                continue
            iou = box_iou(gt_box, pred_box)
            if iou > best_iou:
                best_iou = iou
                best_j = j
        if best_iou >= 0.5:
            matched_gts.add(i)
            matched_preds.add(best_j)
            total_fold_ground_truth += 1
            fold_detections += 1
            total_distance += center_distance(gt_boxes[i][1], pred_boxes[best_j][1])
            count += 1

    unmatched_preds = [j for j in range(len(pred_boxes)) if j not in matched_preds]
    if unmatched_preds:
        false_positive_images.append(label_file.replace('.txt', '.jpg'))

        # === Visualize false positives
        # === Try to locate the actual image file
        image_base = label_file.replace('.txt', '')
        found = False
        for ext in ['.jpg', '.jpeg', '.png']:
            candidate = os.path.join(image_dir, image_base + ext)
            if os.path.exists(candidate):
                image_path = candidate
                found = True
                break

        if not found:
            print(f"⚠️ Could not find image for: {label_file}")
            continue

        img = cv2.imread(image_path)
        if img is not None:
            # Collect boxes and scores for unmatched predictions
            boxes = []
            scores = []
            for j in unmatched_preds:
                _, box = pred_boxes[j]
                x1, y1, x2, y2 = map(int, box)
                w = x2 - x1
                h = y2 - y1
                conf = float(pred_lines[j][5]) if len(pred_lines[j]) >= 6 else 1.0  # fallback to 1.0
                boxes.append([x1, y1, w, h])
                scores.append(conf)

            # Apply NMS to reduce overlapping boxes
            indices = cv2.dnn.NMSBoxes(boxes, scores, score_threshold=0.1, nms_threshold=0.5)
            if indices is not None and len(indices) > 0:
                for i in indices:
                    i = i[0] if isinstance(i, (list, tuple, np.ndarray)) else i
                    x, y, w, h = boxes[i]
                    x2, y2 = x + w, y + h
                    cv2.rectangle(img, (x, y), (x2, y2), (0, 0, 255), 2)  # Red box
                    cv2.putText(img, "FP", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

            output_path = os.path.join(output_fp_dir, label_file.replace('.txt', '.jpg'))
            cv2.imwrite(output_path, img)
        else:
            print(f"⚠️ Could not read image: {image_path}")

    # === Background Images: no predictions
    if not pred_boxes:
        background_images.append(label_file.replace('.txt', '.jpg'))

# === Final Output ===
if count > 0:
    avg_distance = total_distance / count
    print(f"\n📏 Average center distance for detected folds: {avg_distance:.2f} pixels")
    print("\n📈 Plotting and saving sorted center distances...")
    # Collect and sort all matched center distances
    center_distances = []

    for label_file in os.listdir(gt_dir):
        gt_path = os.path.join(gt_dir, label_file)
        pred_path = os.path.join(pred_dir, label_file)
        if not os.path.exists(pred_path):
            continue

        with open(gt_path, 'r') as f:
            gt_lines = [line.strip().split() for line in f.readlines()]
        with open(pred_path, 'r') as f:
            pred_lines = [line.strip().split() for line in f.readlines()]

        gt_boxes = [yolo_to_xyxy(line) for line in gt_lines if int(line[0]) == 0]
        pred_boxes = [yolo_to_xyxy(line) for line in pred_lines if int(line[0]) == 0]

        matched_preds = set()

        for i, (gt_cls, gt_box) in enumerate(gt_boxes):
            best_iou = 0
            best_j = -1
            for j, (pred_cls, pred_box) in enumerate(pred_boxes):
                if j in matched_preds:
                    continue
                iou = box_iou(gt_box, pred_box)
                if iou > best_iou:
                    best_iou = iou
                    best_j = j
            if best_iou >= 0.5:
                matched_preds.add(best_j)
                dist = center_distance(gt_box, pred_boxes[best_j][1])
                center_distances.append(dist)

    center_distances.sort()

    # Create output folder for plots if needed
    plots_dir = "plots"
    os.makedirs(plots_dir, exist_ok=True)
    plot_path = os.path.join(plots_dir, "sorted_center_distances.png")

    # Plot and save
    plt.figure(figsize=(10, 4))
    plt.plot(center_distances, marker='o')
    plt.title("Sorted Center Distance Errors (pixels)")
    plt.xlabel("Match Index (sorted)")
    plt.ylabel("Center Distance")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(plot_path)
    plt.close()

    print(f"✅ Plot saved to: {plot_path}")
else:
    print("📉 No distances to plot (no matched detections).")


Ultralytics 8.3.146 🚀 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX 4000 Ada Generation, 20040MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/student/Desktop/current_model_ai/data.yaml, degrees=15.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=folds_retrain, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, 

train: Scanning /home/student/Desktop/current_model_ai/train/labels.cache... 1184 images, 2 backgrounds, 0 corrupt: 100%|██████████| 1184/1184 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2129.3±1146.0 MB/s, size: 346.7 KB)


val: Scanning /home/student/Desktop/current_model_ai/val/labels.cache... 338 images, 0 backgrounds, 0 corrupt: 100%|██████████| 338/338 [00:00<?, ?it/s]


...


30 epochs completed in 0.053 hours.
Optimizer stripped from /home/student/Desktop/current_model_ai/folder_train/folds_retrain/weights/last.pt, 5.5MB
Optimizer stripped from /home/student/Desktop/current_model_ai/folder_train/folds_retrain/weights/best.pt, 5.5MB

Validating /home/student/Desktop/current_model_ai/folder_train/folds_retrain/weights/best.pt...
Ultralytics 8.3.146 🚀 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX 4000 Ada Generation, 20040MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:01<00:00,  7.60it/s]


                   all        338        510      0.735      0.867      0.842      0.369
Speed: 0.1ms preprocess, 2.3ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /home/student/Desktop/current_model_ai/folder_train/folds_retrain
Ultralytics 8.3.146 🚀 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX 4000 Ada Generation, 20040MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4741.2±1817.9 MB/s, size: 306.1 KB)


val: Scanning /home/student/Desktop/current_model_ai/val/labels.cache... 338 images, 0 backgrounds, 0 corrupt: 100%|██████████| 338/338 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:02<00:00,  8.32it/s]


                   all        338        510      0.732      0.869      0.841      0.367
Speed: 0.3ms preprocess, 2.8ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /home/student/Desktop/current_model_ai/folder_train/folds_retrain2

📏 Average center distance for detected folds: 4.94 pixels

📈 Plotting and saving sorted center distances...
✅ Plot saved to: plots/sorted_center_distances.png


In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

# === Load model (no retraining) ===
model = YOLO('yolo11n.pt')

# === Run validation to get prediction directory ===
val_results = model.val(data='/home/student/Desktop/current_model_ai/data.yaml', save_txt=True)
pred_dir = os.path.join(val_results.save_dir, 'labels')
print(f"📂 Using predictions from: {pred_dir}")

# === Ground truth directory ===
gt_dir = '/home/student/Desktop/current_model_ai/val/labels'
img_w, img_h = 640, 640

# === Helper Functions ===
def yolo_to_xyxy(line, img_w=640, img_h=640):
    cls, x, y, w, h = map(float, line)
    x1 = (x - w / 2) * img_w
    y1 = (y - h / 2) * img_h
    x2 = (x + w / 2) * img_w
    y2 = (y + h / 2) * img_h
    return int(cls), [x1, y1, x2, y2]

def center_distance(box1, box2):
    cx1 = (box1[0] + box1[2]) / 2
    cy1 = (box1[1] + box1[3]) / 2
    cx2 = (box2[0] + box2[2]) / 2
    cy2 = (box2[1] + box2[3]) / 2
    return np.linalg.norm([cx1 - cx2, cy1 - cy2])

def box_iou(box1, box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH
    box1Area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2Area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    unionArea = box1Area + box2Area - interArea
    return interArea / unionArea if unionArea > 0 else 0.0

# === Collect center distances from matched predictions ===
center_distances = []

for label_file in os.listdir(gt_dir):
    gt_path = os.path.join(gt_dir, label_file)
    pred_path = os.path.join(pred_dir, label_file)
    if not os.path.exists(pred_path):
        continue

    with open(gt_path, 'r') as f:
        gt_lines = [line.strip().split() for line in f.readlines()]
    with open(pred_path, 'r') as f:
        pred_lines = [line.strip().split() for line in f.readlines()]

    gt_boxes = [yolo_to_xyxy(line, img_w, img_h) for line in gt_lines if int(line[0]) == 0]
    pred_boxes = [yolo_to_xyxy(line, img_w, img_h) for line in pred_lines if int(line[0]) == 0]

    matched_preds = set()

    for i, (gt_cls, gt_box) in enumerate(gt_boxes):
        best_iou = 0
        best_j = -1
        for j, (pred_cls, pred_box) in enumerate(pred_boxes):
            if j in matched_preds:
                continue
            iou = box_iou(gt_box, pred_box)
            if iou > best_iou:
                best_iou = iou
                best_j = j
        if best_iou >= 0.35:
            matched_preds.add(best_j)
            dist = center_distance(gt_box, pred_boxes[best_j][1])
            center_distances.append(dist)

# === Plot ===
if center_distances:
    center_distances.sort()
    plt.figure(figsize=(10, 4))
    plt.plot(center_distances, marker='o')
    plt.title("Sorted Center Distance Errors")
    plt.xlabel("Match Index (sorted)")
    plt.ylabel("Center Distance (pixels)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No matched predictions with IoU ≥ 0.5.")


Ultralytics 8.3.146 🚀 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX 4000 Ada Generation, 20040MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 6789.9±586.0 MB/s, size: 323.0 KB)


val: Scanning /home/student/Desktop/current_model_ai/val/labels.cache... 338 images, 0 backgrounds, 0 corrupt: 100%|██████████| 338/338 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:06<00:00,  3.19it/s]


                   all        338        510          0          0          0          0
                person        338        510          0          0          0          0
Speed: 0.3ms preprocess, 1.3ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/val4
📂 Using predictions from: runs/detect/val4/labels
⚠️ No matched predictions with IoU ≥ 0.5.


In [15]:
import os
import cv2
import matplotlib.pyplot as plt
import albumentations as A

# Paths
image_name = "Flipped_horizontal_66.png"
image_dir = "/home/student/Desktop/current_model_ai/ImagesTotal"
output_dir = "/home/student/Desktop/current_model_ai/beforeafter"
os.makedirs(output_dir, exist_ok=True)

# Load the image
image_path = os.path.join(image_dir, image_name)
original = cv2.imread(image_path)
original_rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)

# Save original image
cv2.imwrite(os.path.join(output_dir, f"original_{image_name}"), original)

# Define augmentations to apply one at a time
augmentations = {
    "shear=5.0": A.Affine(shear=5.0, p=1.0),
    "scale=0.05": A.Affine(scale=1.05, p=1.0),
    "perspective=0.0": A.Perspective(scale=(0.0, 0.0), p=1.0),
    "translate=0.08": A.Affine(translate_percent=0.08, p=1.0),
    "degrees=0.0": A.Affine(rotate=0.0, p=1.0),
    "hsv (0,0,0)": A.HueSaturationValue(hue_shift_limit=0, sat_shift_limit=0, val_shift_limit=0, p=1.0),
    "flipud=0.0": A.VerticalFlip(p=0.0),
    "fliplr=0.0": A.HorizontalFlip(p=0.0)
}

# Apply and save each augmentation
for name, aug in augmentations.items():
    transformed = aug(image=original)["image"]
    out_name = f"{name.replace('=', '_').replace(' ', '_')}_{image_name}"
    out_path = os.path.join(output_dir, out_name)
    cv2.imwrite(out_path, transformed)

print(f"Saved original and augmented versions of {image_name} in {output_dir}")


Saved original and augmented versions of Flipped_horizontal_66.png in /home/student/Desktop/current_model_ai/beforeafter


In [ ]:
predict_folder = "/home/student/Desktop/current_model_ai/runs/predict"              # Predicted output folder

# Remove the folder if it exists
if os.path.exists(predict_folder):
    shutil.rmtree(predict_folder)

# Load model
model = YOLO('/home/student/Desktop/current_model_ai/folder_train/folds_retrain/weights/best.pt')

# Run predictions
results = model.predict(
    source='/home/student/Desktop/current_model_ai/test/images',
    conf=0.3,
    save=True,
    save_txt=True,
    project='/home/student/Desktop/current_model_ai/runs',
    name='predict',
    save_conf=True  # Save confidence values in the txt output
)


image 1/571 /home/student/Desktop/current_model_ai/test/images/Flipped_horizontal_10.png: 480x640 1 V, 10.2ms
image 2/571 /home/student/Desktop/current_model_ai/test/images/Flipped_horizontal_100.png: 480x640 1 V, 8.5ms
image 3/571 /home/student/Desktop/current_model_ai/test/images/Flipped_horizontal_112.png: 480x640 2 Vs, 9.7ms
image 4/571 /home/student/Desktop/current_model_ai/test/images/Flipped_horizontal_123.png: 480x640 2 Vs, 9.2ms
image 5/571 /home/student/Desktop/current_model_ai/test/images/Flipped_horizontal_124.png: 480x640 3 Vs, 8.3ms
image 6/571 /home/student/Desktop/current_model_ai/test/images/Flipped_horizontal_126.png: 480x640 2 Vs, 9.2ms
image 7/571 /home/student/Desktop/current_model_ai/test/images/Flipped_horizontal_128.png: 480x640 1 V, 8.2ms
image 8/571 /home/student/Desktop/current_model_ai/test/images/Flipped_horizontal_129.png: 480x640 2 Vs, 8.5ms
image 9/571 /home/student/Desktop/current_model_ai/test/images/Flipped_horizontal_13.png: 480x640 1 V, 9.5ms
image

✅ Done! Visualized only center dots and bounding boxes. Pink dots removed.
